# 🧪 Work Overview

In this work, we will:

🧼 Clean and preprocess multiple datasets (elevation, soil, climate, etc.)

🔗 Merge them into a single unified dataset

🔍 Run tests to check whether feature reduction is possible

In [1]:
from scripts.dataMerging.combineDatasets import extract_features_elevation , extract_features_landcover , extract_features_monthly_clim , extract_features_soil , organize_monthly_climat_files
from scripts.dataMerging.mergeDataSources import progressive_merge
from scripts.dataMerging.generateGrid import generate_grid_in_shape
from scripts.dataPreprocessing.dataCleaning import process_fire_data , treat_sensor_errors_soil , impute_with_geo_zones 
from scripts.dataPreprocessing.scalingEncoding import scalingEncodingDataset
from scripts.statistics.firePerSeason import calculate_seasonal_fire_percentage

### 🗺️ Reference Grid

📐 Create a reference grid with consistent latitude and longitude

🔗 Ensures all datasets align and can be merged correctly

In [ ]:

# Step 1: Generate grid (only once)
grid_df = generate_grid_in_shape(
    "../data/shapefiles/combined/alg_tun.shp",
    resolution=0.03, # 1 KM resolution
    output_csv="../data/features/grid_points.csv",
)



📂 Loading shapefile...
🗺️ Bounding box: [-8.67386818 18.96023083 11.98736715 37.55986   ]
📏 Grid: 689 × 620 = 427,180 total potential points
🔍 Filtering points inside region...
✅ 253,077 points inside shapefile
💾 Saved grid to ../data/features/grid_points.csv


## 🔥 Extract Nearest Points (cKDTree)

🌳 Use cKDTree to find the nearest grid point for each fire record

📍 Matches fire locations to the reference grid efficiently

⚡ Fast nearest-neighbor search for large datasets

In [3]:

# Define the paths and parameters
GRID_FILE = "../data/features/grid_points.csv"
FIRE_FILE = "../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv"
TARGET_FIRE_TYPE = 2 

process_fire_data(
    grid_path=GRID_FILE,
    fire_path=FIRE_FILE,
    target_type=TARGET_FIRE_TYPE,
    output_file="../data/preprocessed/fire_preprocessed.csv"
)


✅ Saved (253077, 3) grid points with binary fire data (1/0) to ../data/preprocessed/fire_preprocessed.csv


## ☁️ Climat Dataset

❄️ Extract seasonal data (winter, spring, summer, autumn)

🛠️ Preprocess by fixing missing values using the median Apply regional resolution 

📏 Scale features using a Robust Scaler

In [4]:

# Organize the files
monthly_tmax_data = organize_monthly_climat_files(
    "../data/climate_dataset/5min/max/*.tif"
)
monthly_tmin_data = organize_monthly_climat_files(
    "../data/climate_dataset/5min/min/*.tif"
)
monthly_tprec_data = organize_monthly_climat_files(
    "../data/climate_dataset/5min/prec/*.tif"
)


extract_features_monthly_clim(
    point_csv="../data/features/grid_points.csv",
    raster_dict=monthly_tmax_data,
    output_path="../data/features/grid_tmax.csv",
    col_name="tmax",
)


extract_features_monthly_clim(
    point_csv="../data/features/grid_points.csv",
    raster_dict=monthly_tmin_data,
    output_path="../data/features/grid_tmin.csv",
    col_name="tmin",
)


extract_features_monthly_clim(
    point_csv="../data/features/grid_points.csv",
    raster_dict=monthly_tprec_data,
    output_path="../data/features/grid_tprec.csv",
    col_name="prec",
)

Month 12: 100%|██████████| 253077/253077 [00:46<00:00, 5394.63it/s]


✅ Finished sampling all monthly rasters.
💾 Saved seasonal climatology to ../data/features/grid_tmax.csv


Month 12: 100%|██████████| 253077/253077 [00:42<00:00, 5944.80it/s]


✅ Finished sampling all monthly rasters.
💾 Saved seasonal climatology to ../data/features/grid_tmin.csv


Month 12: 100%|██████████| 253077/253077 [00:32<00:00, 7885.78it/s]


✅ Finished sampling all monthly rasters.
💾 Saved seasonal climatology to ../data/features/grid_tprec.csv


In [5]:
calculate_seasonal_fire_percentage('../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv')


,Season,Count,Percentage
3,Winter,18609,20.62%
2,Spring,23093,25.59%
0,Summer,24667,27.33%
1,Autumn/Fall,23881,26.46%


### 📊 Seasonal Fire Distribution
🔥 As we can see, fires occur almost equally across all seasons

⚠️ Therefore, dropping any season’s climat data is not advisable

In [6]:

impute_with_geo_zones("../data/features/grid_tmax.csv", base_res=0.05 , min_points=10 ,max_res=0.5, output_path="../data/features_cleaned/grid_tmax_clean.csv")
scalingEncodingDataset("../data/features_cleaned/grid_tmax_clean.csv","../data/preprocessed/tmax_preprocessed.csv")

impute_with_geo_zones("../data/features/grid_tmin.csv", base_res=0.05, min_points=10 ,max_res=0.5, output_path="../data/features_cleaned/grid_tmin_clean.csv")
scalingEncodingDataset("../data/features_cleaned/grid_tmin_clean.csv","../data/preprocessed/tmin_preprocessed.csv")


impute_with_geo_zones("../data/features/grid_tprec.csv", base_res=0.05 , min_points=10 ,max_res=0.5, output_path="../data/features_cleaned/grid_prec_clean.csv")
scalingEncodingDataset("../data/features_cleaned/grid_prec_clean.csv","../data/preprocessed/prec_preprocessed.csv")



Missing values (percent) per column :
winter_tmax    0.36076
spring_tmax    0.36076
summer_tmax    0.36076
autumn_tmax    0.36076
dtype: float64

=== Imputing column: winter_tmax ===
winter_tmax: imputation done using geo-zones.

=== Imputing column: spring_tmax ===
spring_tmax: imputation done using geo-zones.

=== Imputing column: summer_tmax ===
summer_tmax: imputation done using geo-zones.

=== Imputing column: autumn_tmax ===
autumn_tmax: imputation done using geo-zones.
💾 Saved imputation to ../data/features_cleaned/grid_tmax_clean.csv
Saved preprocessed dataset → ../data/preprocessed/tmax_preprocessed.csv
Missing values (percent) per column :
winter_tmin    0.36076
spring_tmin    0.36076
summer_tmin    0.36076
autumn_tmin    0.36076
dtype: float64

=== Imputing column: winter_tmin ===
winter_tmin: imputation done using geo-zones.

=== Imputing column: spring_tmin ===
spring_tmin: imputation done using geo-zones.

=== Imputing column: summer_tmin ===
summer_tmin: imputation done 

## 🌳 Landcover Dataset

🌱 Extract landcover values from the reference grid

🛠️ Preprocess by handling missing values using the median Applying regional resolution

📏 Scale features using a Robust Scaler

✅ We only kept the gridcode feature

In [7]:
extract_features_landcover(
    csv_path="../data/features/grid_points.csv",
    shapefile_path="../data/land_dataset/combined/alg_tun_landcvr.shp",
    lat_col="latitude",
    lon_col="longitude",
    keep_cols=["GRIDCODE"],  # can be ["GRIDCODE", "CLASS", "AREA", ...]
    output_path="../data/features/grid_landcover.csv",
)


,latitude,longitude,GRIDCODE
0,18.990231,3.296132,202.0
1,18.990231,3.326132,202.0
2,18.990231,3.356132,202.0
3,18.990231,3.386132,202.0
4,18.990231,3.416132,202.0
...,...,...,...
253072,37.320231,9.776132,14.0
253073,37.320231,9.806132,30.0
253074,37.320231,9.836132,30.0
253075,37.320231,9.866132,210.0


In [8]:
impute_with_geo_zones("../data/features/grid_landcover.csv", base_res=0.05, min_points=10 ,max_res=0.2, output_path="../data/features_cleaned/grid_landcover_clean.csv")
scalingEncodingDataset("../data/features_cleaned/grid_landcover_clean.csv","../data/preprocessed/landcover_preprocessed.csv")


Missing values (percent) per column :
GRIDCODE    0.052948
dtype: float64

=== Imputing column: GRIDCODE ===
GRIDCODE: imputation done using geo-zones.
💾 Saved imputation to ../data/features_cleaned/grid_landcover_clean.csv
Saved preprocessed dataset → ../data/preprocessed/landcover_preprocessed.csv


## 🌱 Soil Dataset

🧱 Extract soil features from the reference grid

🛠️ Preprocess missing and invalid data

Rows with negative values (likely sensor errors) are treated as missing Apply regional resolution

🎨 Feature selection & encoding

TEXTURE_SOTER and TEXTURE_USDA have the same meaning

Keep only TEXTURE_USDA (more detailed)

Apply One-Hot Encoding to TEXTURE_USDA

📏 Scale features using a Robust Scaler

In [9]:
extract_features_soil(
    csv_path="../data/features/grid_points.csv",
    raster_path="../data/soil_dataset/original/HWSD2_RASTER/HWSD2.bil",
    soil_attributes_csv="../data/soil_dataset/simplified/D1_soil_features_alg_tun.csv",
    output_soil_ids="../data/features/fire_soil_ids.csv",
    output_soil_feature="../data/features/grid_soil.csv",
)

(         latitude  longitude  HWSD2_SMU_ID
 0       18.990231   3.296132          1796
 1       18.990231   3.326132          1796
 2       18.990231   3.356132          1796
 3       18.990231   3.386132          1796
 4       18.990231   3.416132          1796
 ...           ...        ...           ...
 253072  37.320231   9.776132         31828
 253073  37.320231   9.806132         31828
 253074  37.320231   9.836132         31828
 253075  37.320231   9.866132          7001
 253076  37.530231   8.936132         31802
 
 [253077 rows x 3 columns],
          latitude  longitude  COARSE  SAND  SILT  CLAY  TEXTURE_USDA  \
 0       18.990231   3.296132    12.0  57.0  27.0  16.0          11.0   
 1       18.990231   3.326132    12.0  57.0  27.0  16.0          11.0   
 2       18.990231   3.356132    12.0  57.0  27.0  16.0          11.0   
 3       18.990231   3.386132    12.0  57.0  27.0  16.0          11.0   
 4       18.990231   3.416132    12.0  57.0  27.0  16.0          11.0   
 ...

In [ ]:
import pandas as pd
df = pd.read_csv("../data/features/grid_soil.csv")
if "TEXTURE_SOTER" in df.columns:
        df.drop(columns=["TEXTURE_SOTER"], inplace=True)

In [2]:
treat_sensor_errors_soil("../data/features/grid_soil.csv",output_path="../data/features/grid_soil_treated.csv")

✔ Cleaning complete!
  Deleted rows : 44349
  Fixed rows   : 59021


In [ ]:


CATEGORICAL_COLS_SOIL = ["TEXTURE_USDA"]  # categorical columns
NUMERIC_COLS_SOIL = [
    "COARSE", "SAND", "SILT", "CLAY", "BULK", "REF_BULK", "ORG_CARBON", "PH_WATER",
    "TOTAL_N", "CN_RATIO", "CEC_SOIL", "CEC_CLAY", "CEC_EFF", "TEB", "BSAT",
    "ALUM_SAT", "ESP", "TCARBON_EQ", "GYPSUM", "ELEC_COND"
]  # numeric columns

# Usage
soil_cleaned = impute_with_geo_zones("../data/features/grid_soil_treated.csv",num_cols=NUMERIC_COLS_SOIL , cat_cols=CATEGORICAL_COLS_SOIL,  base_res=0.05, min_points=10 ,max_res=0.5, output_path="../data/features_cleaned/grid_soil_clean.csv")


Missing values (percent) per column :
COARSE          11.776513
SAND            11.776513
SILT            11.776513
CLAY            11.776513
TEXTURE_USDA    11.776513
BULK            11.776513
REF_BULK        11.776513
ORG_CARBON      11.776513
PH_WATER        11.776513
TOTAL_N         11.776513
CN_RATIO        11.776513
CEC_SOIL        11.776513
CEC_CLAY        11.776513
CEC_EFF         11.776513
TEB             11.776513
BSAT            11.776513
ALUM_SAT        11.776513
ESP             11.776513
TCARBON_EQ      11.776513
GYPSUM          11.776513
ELEC_COND       11.776513
dtype: float64

=== Imputing column: COARSE ===


In [ ]:
scalingEncodingDataset("../data/features_cleaned/grid_soil_clean.csv","../data/preprocessed/soil_preprocessed.csv",categorical_col=["TEXTURE_USDA"])


TEXTURE_USDA classes found: [np.int64(3), np.int64(5), np.int64(7), np.int64(9), np.int64(10), np.int64(11), np.int64(12)]
Saved preprocessed dataset → ../data/preprocessed/soil_preprocessed.csv


## 🏔️ Elevation Dataset

🗻 Extract elevation values from the reference grid

🛠️ Preprocess by handling missing values using the median

🌍 Apply regional resolution if needed

📏 Scale features using a Robust Scaler

In [ ]:
fires_with_elevation = extract_features_elevation(
    raster_path="../data/elevation_dataset/simplified/elevation_clipped.tif",
    fire_csv_path="../data/features/grid_points.csv",
    output_csv="../data/features/grid_elevation.csv",
    value_name="elevation",
)


Loaded 91102 points from ../data/features/grid_points.csv


Extracting elevation: 100%|██████████| 91102/91102 [00:13<00:00, 6933.98it/s]


✅ Saved extracted elevation to ../data/features/grid_elevation.csv


In [ ]:
impute_with_geo_zones("../data/features/grid_elevation.csv", base_res=0.05, min_points=10 ,max_res=0.5, output_path="../data/features_cleaned/grid_elevation_clean.csv")
scalingEncodingDataset("../data/features_cleaned/grid_elevation_clean.csv","../data/preprocessed/elevation_preprocessed.csv")


Missing values (percent) per column :
Series([], dtype: float64)
Saved preprocessed dataset → ../data/preprocessed/elevation_preprocessed.csv


## 📊 Merging Preprocessed Datasets
Merging all preprocessed datasets on the common key fields of longitude and latitude to obtain one final, unified dataset for analysis.

In [ ]:
csv_list= ["../data/preprocessed/tmax_preprocessed.csv", "../data/preprocessed/tmin_preprocessed.csv","../data/preprocessed/prec_preprocessed.csv",  "../data/preprocessed/landcover_preprocessed.csv" , "../data/preprocessed/elevation_preprocessed.csv" , "../data/preprocessed/soil_preprocessed.csv","../data/preprocessed/fire_preprocessed.csv"]
temp_df = progressive_merge(
    csv_list,
    on=["latitude", "longitude"],
    how="inner",
    output_path="../data/Merged/merged.csv"
)


Loading first CSV: ../data/features_cleaned/grid_tmax_clean.csv
🔁 Merging file 2/7: ../data/features_cleaned/grid_tmin_clean.csv
✅ Intermediate merged size: (91102, 10)
🔁 Merging file 3/7: ../data/features_cleaned/grid_tprec_clean.csv
✅ Intermediate merged size: (91102, 11)
🔁 Merging file 4/7: ../data/features_cleaned/grid_landcover_clean.csv
✅ Intermediate merged size: (91102, 12)
🔁 Merging file 5/7: ../data/features_cleaned/grid_elevation_clean.csv
✅ Intermediate merged size: (91102, 13)
🔁 Merging file 6/7: ../data/features_cleaned/grid_soil_clean.csv
✅ Intermediate merged size: (196405, 35)
🔁 Merging file 7/7: ../data/features_cleaned/grid_fire_clean.csv
✅ Intermediate merged size: (196405, 36)
✅ All files merged successfully.
